In [1]:
import cv2
import time
import imutils
import argparse
import numpy as np

from imutils.video import FPS
from imutils.video import VideoStream

In [9]:
#Constructing Argument Parse to input from Command Line
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required=True,default=r"./Caffe/street.jpg",help="image")
ap.add_argument("-p", "--prototxt", required=True,default=r"./Caffe/SSD_MobileNet_prototxt.txt",help="prototxt")
ap.add_argument("-m", "--model", required=True,default=r"./Caffe/SSD_MobileNet.caffemodel",help="model")
ap.add_argument("-c", "--confidence", type=float, default=0.2,help="minimum probability to filter weak detections")
args = vars(ap.parse_args())

usage: ipykernel_launcher.py [-h] -i IMAGE -p PROTOTXT -m MODEL
                             [-c CONFIDENCE]
ipykernel_launcher.py: error: the following arguments are required: -i/--image, -p/--prototxt, -m/--model


SystemExit: 2

In [6]:
labels = ["background", "aeroplane", "bicycle", "bird", 
"boat","bottle", "bus", "car", "cat", "chair", "cow", 
"diningtable","dog", "horse", "motorbike", "person", "pottedplant", 
"sheep","sofa", "train", "tvmonitor"]
colors = np.random.uniform(0, 255, size=(len(labels), 3))

In [7]:
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


NameError: name 'args' is not defined

In [ ]:
image = cv2.imread(args["image"])
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(cv2.resize(image, (300, 300)), 0.007843,
	(300, 300), 127.5)

In [ ]:
print("[INFO] computing object detections...")
net.setInput(blob)
detections = net.forward()

In [ ]:
for i in np.arange(0, detections.shape[2]):
	# extract the confidence (i.e., probability) associated with the
	# prediction
	confidence = detections[0, 0, i, 2]
	# filter out weak detections by ensuring the `confidence` is
	# greater than the minimum confidence
	if confidence > args["confidence"]:
		# extract the index of the class label from the `detections`,
		# then compute the (x, y)-coordinates of the bounding box for
		# the object
		idx = int(detections[0, 0, i, 1])
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")
		# display the prediction
		label = "{}: {:.2f}%".format(CLASSES[idx], confidence * 100)
		print("[INFO] {}".format(label))
		cv2.rectangle(image, (startX, startY), (endX, endY),
			COLORS[idx], 2)
		y = startY - 15 if startY - 15 > 15 else startY + 15
		cv2.putText(image, label, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.5, COLORS[idx], 2)

In [ ]:
cv2.imshow("Output", image)
cv2.waitKey(0)